In [ ]:
import pandas as pd
import pickle
import requests
import json
import time

# Verification of complete information

# SAoT prompt

In [ ]:
def callLLM_SAoT(model,dfg,ind):
    prompt = '''
                You are an expert in Recognizing Textual Entailment over pairs of Premise and Hypothesis.
        Classify the relationship between the given Premise and Hypothesis as one of the following: "Entailment", "Neutral" or "Contradiction". 

        Premise: '''+dfg.at[ind,"Texto"]+'''
        Hypothesis: '''+ dfg.at[ind,"Hipotesis"]+'''

        Let's think step by step

        Step 1: Identify all the relationships of Generalization, Equivalence, Opposition, Concretization or Specificity between terms from the premise to the hypothesis. This process is performed for each of the terms in the hypothesis.
        Use the next format for relationships: (p_i,rel,h_j) where p_i is a term from the premise, h_j is a term from the hypothesis, and rel is the relation linking these terms. 
        If any terms of the hypothesis with an unknown relationship with terms in the premise, identify them as (,unknown,h_k) where h_k is in the hypothesis. 
        List the relationships found.

        Step 2: Align all the relationships found with the NLI labels: Entailment, Neutral, Contradiction, classifying them into groups G1, G2, G3 and G4 according to the following:
        G1: will contain the list of generalization or equivalence relationships that usually correspond to Entailment.
        G2: will contain the list of relationships of opposition that often correspond to Contradiction.
        G3: will contain the list of relationships of concretization or specificity that typically correspond to Neutral.
        G4: will contain the list of terms of the hypothesis with an unknown relationship with the premise. 

        Step 3: Analyze each group of relationships, weigh the groups obtained that support your decision, and classify the correct label for the premise and hypothesis presented.

        Fill in the template:
        {
        "G1":[],
        "G2":[],
        "G3":[],
        "G4":[],
        "Answer": "",
        "Explanation": ""
        }
        Respond only using the template.
            '''
    
    #print(prompt)
    # break
    data = {
        "prompt": prompt,
        "model": model,
        "format": "json",
        "stream": False,
        "options": {"temperature": 0,
                    "num_ctx":4196},
    }
    try:
        response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=1980)      
        json_data = json.loads(response.text)
        return True,json.dumps(json.loads(json_data["response"]), indent=2)
    except:
        print("Error")
        #print(response.text)
    #print(response.text)
    return False,str(response.text)

# Variables

In [ ]:
m_n=14
ruta="../output2/relationships/"
modelos=["gemma2","gemma2:2b","llama3.1","llama3.2","phi3:medium","phi3"]
corpus=["Scitail","RTEGLUE","SNLI","SICK"]

# SAoT

In [ ]:
conjunto_etiquetas=set()
for m in modelos:
    for c in corpus:
        print("Model",m,"Corpus",c)
        for i in range(m_n):
            new_data = {'Answer':[]}#,'Explanation':[]} # 
            mt = pd.read_pickle(ruta+c+"/"+c.lower()+str(i+1)+".pickle")
            with open(m+"/"+c+"/processed/ritB_SAoT"+str(i+1)+".pickle", "rb") as f:
                lista_respuestas = pickle.load(f)
            # Validar si faltan respuestas
            j=0
            for a in lista_respuestas:
                try:
                    temp_dict=json.loads(a)
                    resp=temp_dict["Answer"]
                    if resp=="" or resp=="NA":
                        raise Exception("Nothing")
                    #exp=temp_dict["Explanation"]
                    else:
                        new_data["Answer"].append(resp)
                    #new_data["Explanation"].append(exp)
                except:
                    print(j)
                    try:
                        ex,dat=callLLM_SAoT(m,mt,j)
                        if ex==False:
                            if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat or '\"Answer\": \"NEUTRAL\"' in dat or '\\"Answer\\": \\"NEUTRAL\\"' in dat):
                                new_data["Answer"].append("Neutral")
                            elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat or '\"Answer\": \"ENTAILMENT\"' in dat or '\\"Answer\\": \\"ENTAILMENT\\"' in dat):
                                new_data["Answer"].append("Entailment")
                            elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat or '\"Answer\": \"CONTRADICTION\"' in dat or '\\"Answer\\": \\"CONTRADICTION\\"' in dat):
                                new_data["Answer"].append("Contradiction")
                            else:
                                time.sleep(250)
                                try:
                                    ex,dat=callLLM_SAoT(m,mt,j)
                                    if ex==False:
                                        if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat or '\"Answer\": \"NEUTRAL\"' in dat or '\\"Answer\\": \\"NEUTRAL\\"' in dat):
                                            new_data["Answer"].append("Neutral")
                                        elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat or '\"Answer\": \"ENTAILMENT\"' in dat or '\\"Answer\\": \\"ENTAILMENT\\"' in dat):
                                            new_data["Answer"].append("Entailment")
                                        elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat or '\"Answer\": \"CONTRADICTION\"' in dat or '\\"Answer\\": \\"CONTRADICTION\\"' in dat):
                                            new_data["Answer"].append("Contradiction")
                                        else:
                                            new_data["Answer"].append("")
                                            print("F")
                                        #new_data["Explanation"].append(dat)
                                        print("a la segunda")
                                    else:
                                        temp_dict=json.loads(dat)
                                        resp=temp_dict["Answer"]
                                        #exp=temp_dict["Explanation"]
                                        new_data["Answer"].append(resp)
                                        #new_data["Explanation"].append(exp)
                                        print("Hecho")
                                except:
                                    new_data["Answer"].append("NA")
                                    #new_data["Explanation"].append("")
                                    print("a la tercera")
                        else:
                            temp_dict=json.loads(dat)
                            resp=temp_dict["Answer"]
                            #exp=temp_dict["Explanation"]
                            new_data["Answer"].append(resp)
                            #new_data["Explanation"].append(exp)
                            print("Hecho")
                    except:
                        new_data["Answer"].append("NA")
                        #new_data["Explanation"].append("")
                        print("a la tercera")
                j+=1
            pd.DataFrame(new_data).to_pickle("../LLMs2/"+m+"/"+c+"/complete/ritB_SAoT"+str(i+1)+".pickle")
            print("../LLMs2/"+m+"/"+c+"/complete/ritB_SAoT"+str(i+1)+".pickle")
            

In [ ]:
m_n=1
ruta="../output2/relationships/"
modelos=["gemma2","gemma2:2b","llama3.1","llama3.2","phi3:medium","phi3"]
corpus=["DIAG"]

In [ ]:
conjunto_etiquetas=set()
for m in modelos:
    for c in corpus:
        print("Model",m,"Corpus",c)
        for i in range(m_n):
            new_data = {'Answer':[]}#,'Explanation':[]} # 
            mt = pd.read_pickle(ruta+c+"/"+c.lower()+str(i+1)+".pickle")
            with open(m+"/"+c+"/processed/ritB_SAoT"+str(i+1)+".pickle", "rb") as f:
                lista_respuestas = pickle.load(f)
            # Validar si faltan respuestas
            j=0
            for a in lista_respuestas:
                try:
                    temp_dict=json.loads(a)
                    resp=temp_dict["Answer"]
                    if resp=="" or resp=="NA":
                        raise Exception("Nothing")
                    #exp=temp_dict["Explanation"]
                    else:
                        new_data["Answer"].append(resp)
                    #new_data["Explanation"].append(exp)
                except:
                    print(j)
                    try:
                        ex,dat=callLLM_SAoT(m,mt,j)
                        if ex==False:
                            if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat or '\"Answer\": \"NEUTRAL\"' in dat or '\\"Answer\\": \\"NEUTRAL\\"' in dat):
                                new_data["Answer"].append("Neutral")
                            elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat or '\"Answer\": \"ENTAILMENT\"' in dat or '\\"Answer\\": \\"ENTAILMENT\\"' in dat):
                                new_data["Answer"].append("Entailment")
                            elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat or '\"Answer\": \"CONTRADICTION\"' in dat or '\\"Answer\\": \\"CONTRADICTION\\"' in dat):
                                new_data["Answer"].append("Contradiction")
                            else:
                                time.sleep(250)
                                try:
                                    ex,dat=callLLM_SAoT(m,mt,j)
                                    if ex==False:
                                        if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat or '\"Answer\": \"NEUTRAL\"' in dat or '\\"Answer\\": \\"NEUTRAL\\"' in dat):
                                            new_data["Answer"].append("Neutral")
                                        elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat or '\"Answer\": \"ENTAILMENT\"' in dat or '\\"Answer\\": \\"ENTAILMENT\\"' in dat):
                                            new_data["Answer"].append("Entailment")
                                        elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat or '\"Answer\": \"CONTRADICTION\"' in dat or '\\"Answer\\": \\"CONTRADICTION\\"' in dat):
                                            new_data["Answer"].append("Contradiction")
                                        else:
                                            new_data["Answer"].append("")
                                            print("F")
                                        #new_data["Explanation"].append(dat)
                                        print("a la segunda")
                                    else:
                                        temp_dict=json.loads(dat)
                                        resp=temp_dict["Answer"]
                                        #exp=temp_dict["Explanation"]
                                        new_data["Answer"].append(resp)
                                        #new_data["Explanation"].append(exp)
                                        print("Hecho")
                                except:
                                    new_data["Answer"].append("NA")
                                    #new_data["Explanation"].append("")
                                    print("a la tercera")
                        else:
                            temp_dict=json.loads(dat)
                            resp=temp_dict["Answer"]
                            #exp=temp_dict["Explanation"]
                            new_data["Answer"].append(resp)
                            #new_data["Explanation"].append(exp)
                            print("Hecho")
                    except:
                        new_data["Answer"].append("NA")
                        #new_data["Explanation"].append("")
                        print("a la tercera")
                j+=1
            pd.DataFrame(new_data).to_pickle("../LLMs2/"+m+"/"+c+"/complete/ritB_SAoT"+str(i+1)+".pickle")
            print("../LLMs2/"+m+"/"+c+"/complete/ritB_SAoT"+str(i+1)+".pickle")
            